# 1.Auxiliary segmentation (AuxSeg): an interactive tool for ROI segmentation

AuxSeg is designed to select the regions of interest (e.g., an entire tissue or a tumor core) in any 
arbitrary geometrical shape based on simple mouse clicks (instead of a pure-manual boundary drawing).

**AuxSeg is implemented with three modes:**

### (1) Livewire mode (a simplified magic wand tool in Photoshop). 

This allows the user to compute a ‘best’ path, with a minimum color gradient cost (see technicals), 
from a seed point to the next seed point. When these seed points land on the boundaries of a tissue 
in sequence, the livewire will snap on the tissue boundaries in the image.

Related mouse and keyboard operation:
1. Plot a seed point: Left mouse button single click;
2. Draw a line: Mouse movement. In the livewire mode, the predicted path will be pre-showed on the image.

![jupyter](./screenshot/1_livewire_demo.png)

### (2) Straight-line mode. 
This allows the user to draw the straight line between two seed points, when the livewire does not 
give the accurate prediction along some fuzzy edges, or the use just want to generate a straight 
regular border.

Related mouse and keyboard operation:
1. Switch to straight-line mode: Keyboard ‘s’.

![jupyter](./screenshot/2_straightline_demo.png)

### (3) Withdraw mode. 
This allows the user to cancel the last operation.

Related mouse and keyboard operation:
1. Withdraw: Keyboard ‘z’.

**See our breif tutorial video here**: https://www.dropbox.com/s/oy87fpazivg85iw/livewire_tissue_seg.mp4?dl=0

## Code demo

In [ ]:
# import packages
import spateo as st
import numpy as np
import matplotlib.pyplot as plt


# raw binning data, which contains spot/bins from the whole chip
# bin50_h5ad: https://www.dropbox.com/s/vtapwsccpi885l2/mousebrain_bin50_raw.h5ad?dl=0
fname_bin50 = "mousebrain_bin50_raw.h5ad"
adata_bin50 = st.sample_data.mousebrain(fname_bin50)


# rescale spatial coordinates to generate UMI image
coor = np.column_stack((adata_bin50.obsm['spatial'], adata_bin50.X.sum(axis=1).A)).astype('int')
coor[:, 0] = ((coor[:, 0] - 25) / 50).astype('int')
coor[:, 1] = ((coor[:, 1] - 25) / 50).astype('int')
img = np.zeros(shape=[max(coor[:, 0]) + 1, max(coor[:, 1]) + 1], dtype='int')
for line in coor:
    img[line[0], line[1]] = 20 if line[2] > 2000 else int(line[2] / 100)


# perform interactive tissue cut (interactive process, see tutorial video)
tissue_seg = st.pp.auxseg.LiveWireSegmentation(img, smooth_image=False, threshold_gradient_image=False)
plt.scatter(coor[:, 1], coor[:, 0], c=coor[:, 2], s=30)
tissue_seg.connect()
plt.show()


# subset adata to tissue area only
tissue_area = [x.tolist() in tissue_seg.rst.tolist() for x in adata_bin50.obsm['spatial'].astype('int')]
adata_bin50 = adata_bin50[tissue_area,: ]
adata_bin50